# Example usage of XTB via PPQM

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import logging
import sys

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem.Draw import MolsToGridImage

In [ ]:
import ppqm

## Set logging level

In [ ]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger("ppqm").setLevel(logging.INFO)
logging.getLogger("xtb").setLevel(logging.DEBUG)
show_progress = False

## Define a molecule you like

In [ ]:
smiles = "Cc1cc(NCCO)nc(-c2ccc(Br)cc2)n1"  # CHEMBL1956589
molobj = Chem.MolFromSmiles(smiles)

In [ ]:
molobj

## Get some 3D conformers (RDKit)

In [ ]:
molobj = ppqm.tasks.generate_conformers(molobj)

In [ ]:
molobj.GetNumConformers()

## Set XTB Settings

In [ ]:
xtb_options = {
    "scr": "./_tmp_directory_",  # Where should the calculations happen?
    "cmd": "xtb",  # What is the executable?
    "n_cores": 8,  # How many cores to use?
    "show_progress": show_progress,
}

In [ ]:
calc = ppqm.xtb.XtbCalculator(**xtb_options)

In [ ]:
calc

## Calculate some properties

We can now define calculation options and get XTB properties for each conformer in the molobj.

The options for XTB follows the --arg documentation from the xtb website https://xtb-docs.readthedocs.io/en/latest/commandline.html


In [ ]:
# Optimize molecule in water
optimize_options = {
    "gfn": 2,
    "alpb": "h2o",
    "opt": None,
}

# Calculate fukui coefficients
fukui_options = {
    "gfn": 2,
    "alpb": "h2o",
    "vfukui": None,
}


Optimize molobj and set resulting coordinates. The `.calculate` function will return a `List[Dict]` with a property dictionary for each conformer.

In [ ]:
optimize_results = calc.calculate(molobj, optimize_options)

In [ ]:
for idx, prop in enumerate(optimize_results):

    energy = prop["scc_energy"]  # AU
    energy *= ppqm.units.hartree_to_kcalmol
    coord = prop["coord"]

    print(idx, energy)
    ppqm.chembridge.molobj_set_coordinates(molobj, coord, confid=idx)

In [ ]:
df_optimize = pd.DataFrame(optimize_results)

In [ ]:
df_optimize

## Use nglviewer to see your molecules

In [ ]:
# TODO

## Use the updated coordinates to calculate fukui properties

Again, a list of properties (one for each conformer) is returned


In [ ]:
fukui_results = calc.calculate(molobj, fukui_options)

In [ ]:
df_fukui = pd.DataFrame(fukui_results)

In [ ]:
df